In [1]:
agg_columns = ['company_id', 'departure_time']
operations = {"flight_id":"COUNT"}
time_column = "departure_time"
time = "WEEKDAY"

In [3]:
from create_fact_table_generator import TableConverterAgent
from response_generator import ModelResponseAgent

In [ ]:
agent = TableConverterAgent()
res = agent.start_fact_table(agg_columns, operations, time_column, time)

In [33]:
response_agent = ModelResponseAgent()
res = response_agent.ask_question("What are the companies with the most flights on Friday?")
print(res['final_result'])
print(res['decision'])

{ 'RayanAir', 'Turkish Airlines', 'Airways Co' }  
{ 6, 3, 2 }  
{ Airlines }  
{ Number of Flights }  
{ Companies with the Most Flights on Friday }
chart


In [ ]:
import sqlite3
import ast
import os

def parse_question(user_question): 
    BASE_DIR = os.path.abspath(os.path.join(os.path.dirname(__file__), "../../../"))
    DB_PATH = os.path.join(BASE_DIR, "data.db")
    connection = sqlite3.connect(DB_PATH)
    cursor = connection.cursor()

    cursor.execute("SELECT count(*) FROM sqlite_master WHERE type='table';")
    table_count = cursor.fetchone()[0]

    if table_count == 0:
        return ("You haven't uploaded any files yet", "text")
    else:
        agent = ModelResponseAgent()
        state = agent.ask_question(user_question)
        result = state['final_result']
        type = state['decision']
        if state == "chart":
            result = parse_chart_output(result)
        return (result, type)
    
def parse_chart_output(output: str):
    lines = [line.strip().strip(';') for line in output.strip().splitlines()]
    lines = [line for line in lines if line and not line.startswith('```')]  # remove markdown block

    if len(lines) < 5:
        raise ValueError("Invalid output format: not enough lines")

    try:
        x_values = ast.literal_eval(lines[0])
        y_values = ast.literal_eval(lines[1])
        x_label = lines[2]
        y_label = lines[3]
        title = lines[4]
    except Exception as e:
        raise ValueError(f"Error parsing chart output: {e}")

    return x_values, y_values, x_label, y_label, title

(res, type) = parse_question("What are the companies with the most flights on Friday?")
print(type)
print(res)

ModuleNotFoundError: No module named 'utils'